In [1]:
import math
import os.path as path
import numpy as np
import pandas as pd
from skimage.util.montage import montage2d
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D, BatchNormalization, Dropout, MaxPooling2D, Dense, Flatten, ZeroPadding2D, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.layers import average, Input, Concatenate
from extra_functions import *
from scipy import signal


def load_and_format(in_path):
    out_df = pd.read_json(in_path)
    out_images = out_df.apply(lambda c_row: [np.stack([c_row['band_1'],c_row['band_2']], -1).reshape((75,75,2))],1)
    out_images = np.stack(out_images).squeeze()
    return out_df, out_images

dir_path = path.abspath(path.join('__file__',"../.."))
train_path = dir_path + "/train.json"
test_path = dir_path + "/test.json"

train_df, train_images = load_and_format(train_path)
test_df, test_images = load_and_format(test_path)
x_angle_test = np.array(test_df.inc_angle)   
    
"""
x_train, x_val, x_angle_train, x_angle_val, y_train, y_val = train_test_split(train_images
                    , x_angle_train, to_categorical(train_df['is_iceberg']), random_state=2017, train_size=0.75)

print('Train', x_train.shape, y_train.shape)
print('Validation', x_val.shape, y_val.shape) """


print("reading data")
x_train = pd.read_csv('x_train.csv', header=None)
x_train = x_train.values
y_train = pd.read_csv('y_train.csv', header=None)
y_train = y_train.values
x_val = pd.read_csv('x_val.csv', header=None)
x_val = x_val.values
y_val = pd.read_csv('y_val.csv', header=None)
y_val = y_val.values  

print("reshaping data")
x_train = np.reshape(x_train, (3366, 75, 75, 2))
x_val = np.reshape(x_val, (482, 75, 75, 2))

weight_decay = 0.005

image_input = Input(shape=(75, 75, 2), name="image")

cnn = BatchNormalization(momentum=0.99)(image_input)

#0.3
cnn = Conv2D(64, kernel_size=(3,3), padding = 'same')(cnn)
cnn = Activation('relu')(cnn)
cnn = BatchNormalization(momentum=0.99)(cnn)
cnn = MaxPooling2D(pool_size=(2,2))(cnn)
cnn = Dropout(0.25)(cnn)

#0.3
cnn = Conv2D(64, kernel_size=(3,3), padding = 'same')(cnn)
cnn = Activation('relu')(cnn)
cnn = BatchNormalization(momentum=0.99)(cnn)
cnn = MaxPooling2D(pool_size=(2,2))(cnn)
cnn = Dropout(0.25)(cnn)

#0.3
cnn = Conv2D(128, kernel_size=(3,3), padding = 'same')(cnn)
cnn = Activation('relu')(cnn)
cnn = BatchNormalization(momentum=0.99)(cnn)
cnn = MaxPooling2D(pool_size=(2,2))(cnn)
cnn = Dropout(0.25)(cnn)

#0.3
cnn = Conv2D(128, kernel_size=(3,3), padding = 'same')(cnn)
cnn = Activation('relu')(cnn)
cnn = BatchNormalization(momentum=0.99)(cnn)
cnn = MaxPooling2D(pool_size=(2,2))(cnn)
cnn = Dropout(0.25, seed=25)(cnn)

#0.4
cnn = Conv2D(256, kernel_size=(3,3), padding = 'same')(cnn)
cnn = Activation('relu')(cnn)
cnn = BatchNormalization(momentum=0.99)(cnn)
cnn = MaxPooling2D(pool_size=(2,2))(cnn)
cnn = Dropout(0.25, seed=25)(cnn)

#0.4
cnn = Conv2D(512, kernel_size=(3,3), padding = 'same')(cnn)
cnn = Activation('relu')(cnn)
cnn = BatchNormalization(momentum=0.99)(cnn)
cnn = MaxPooling2D(pool_size=(2,2))(cnn)
cnn = Dropout(0.25, seed=25)(cnn)

cnn = Flatten()(cnn)

#256
cnn = Dense(256, activation='relu')(cnn)
cnn = Dropout(0.4, seed=25)(cnn)

#75
cnn = Dense(75, activation='relu')(cnn)
cnn = Dropout(0.5, seed=25)(cnn)

output = Dense(2, activation='softmax')(cnn)


model = Model(inputs=image_input, outputs=output)
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()
print("Training")
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20)
model.fit(x_train, y_train, batch_size = 64, validation_data = (x_val, y_val), 
          epochs = 250, shuffle = True, callbacks=[early_stopping])


print("predicting")
test_predictions = model.predict(test_images)

pred_df = test_df[['id']].copy()
pred_df['is_iceberg'] = test_predictions[:,1]
print("creating csv")
pred_df.to_csv('predictions_5.csv', index = False)

Using TensorFlow backend.


reading data
reshaping data
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           (None, 75, 75, 2)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 75, 75, 2)         8         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 64)        1216      
_________________________________________________________________
activation_1 (Activation)    (None, 75, 75, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 75, 75, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 37, 37, 64) 

3366/3366 [==============================] - 6s - loss: 0.2461 - acc: 0.9058 - val_loss: 0.3204 - val_acc: 0.8465
Epoch 23/250
3366/3366 [==============================] - 6s - loss: 0.2223 - acc: 0.9100 - val_loss: 0.4092 - val_acc: 0.8506
Epoch 24/250
3366/3366 [==============================] - 6s - loss: 0.2129 - acc: 0.9124 - val_loss: 0.3650 - val_acc: 0.8631
Epoch 25/250
3366/3366 [==============================] - 6s - loss: 0.2137 - acc: 0.9133 - val_loss: 0.3103 - val_acc: 0.8485
Epoch 26/250
3366/3366 [==============================] - 6s - loss: 0.2203 - acc: 0.9070 - val_loss: 0.3632 - val_acc: 0.8651
Epoch 27/250
3366/3366 [==============================] - 6s - loss: 0.2206 - acc: 0.9091 - val_loss: 0.7176 - val_acc: 0.7905
Epoch 28/250
3366/3366 [==============================] - 5s - loss: 0.1932 - acc: 0.9180 - val_loss: 0.2554 - val_acc: 0.8880
Epoch 29/250
3366/3366 [==============================] - 5s - loss: 0.1944 - acc: 0.9234 - val_loss: 0.2993 - val_acc: 0.87